Syntax natürlicher Sprachen, WS 2023/24

# 07 - Übung (Lösung)

In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Analyse komplexer Satzkonstruktionen

### a) Schreiben Sie eine CFG, so dass für den folgende ambigen Satz zwei linguistisch korrekte Syntaxanalysen ausgegeben werden. Geben Sie auch die entsprechenden Dependenzanalysen für die beiden Strukturanalysen an.

- *Haustiere denken aber wilde Tiere jagen*



In [3]:
sent = 'Haustiere denken aber wilde Tiere jagen'

#### Konstituentenanalyse:

In [4]:
grammar = nltk.CFG.fromstring("""
    S -> S CC S
    VP -> V CC SBAR
    SBAR -> COMP S

    S -> NP VP
    NP ->  N | ADJ N
    VP -> V | V NP

    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'denken' | 'jagen'
    CC -> 'aber'
    COMP ->  
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                      S                        
           ┌──────────┼───────────────┐         
           S          │               S        
    ┌──────┴────┐     │          ┌────┴─────┐   
    NP          VP    │          NP         VP 
    │           │     │     ┌────┴────┐     │   
    N           V     CC   ADJ        N     V  
    │           │     │     │         │     │   
Haustiere     denken aber wilde     Tiere jagen

            S                                    
    ┌───────┴──────────┐                          
    │                  VP                        
    │       ┌─────┬────┴──────────┐               
    │       │     │              SBAR            
    │       │     │    ┌──────────┴─────┐         
    │       │     │    │                S        
    │       │     │    │          ┌─────┴─────┐   
    NP      │     │    │          NP          VP 
    │       │     │    │     ┌────┴─────┐     │   
    N       V     CC  COMP  ADJ         N     V  
    │   

#### Dependenzanalyse:

In [5]:
# Analyse 1: Koordination von 2 Hauptsätzen (conj)
sent_nr = """
1 Haustiere 2 nsubj
2 denken 0 ROOT
3 aber 7 cc
4 (dass) 7 (mark)
5 wilde 6 amod
6 Tiere 7 nsubj
7 jagen 2 conj
"""
show_dep_trees(sent_nr)

In [6]:
# Analyse 2: subordinierter Satz (Objektsatz ccomp)
sent_nr = """
1 Haustiere 2 nsubj
2 denken 0 ROOT
3 aber 2 cc
4 wilde 5 amod
5 Tiere 6 nsubj
6 jagen 2 ccomp
"""
show_dep_trees(sent_nr)

### b) Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Haustiere hoffen wilde Tiere zu jagen*


In [7]:
sent = 'Haustiere hoffen wilde Tiere zu jagen'

#### Konstituentenanalyse:

In [90]:
grammar = nltk.CFG.fromstring(
    """
    S -> NP VP
    VP -> V VP-INF
    VP-INF -> NP COMP V

    NP ->  N | ADJ N

    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'hoffen' | 'jagen'
    COMP -> 'zu'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

            S                               
    ┌───────┴──────┐                         
    │              VP                       
    │       ┌──────┴─────────┐               
    │       │              VP-INF           
    │       │           ┌────┴─────┬─────┐   
    NP      │           NP         │     │  
    │       │      ┌────┴────┐     │     │   
    N       V     ADJ        N    COMP   V  
    │       │      │         │     │     │   
Haustiere hoffen wilde     Tiere   zu  jagen



#### Dependenzanalyse:

In [11]:
# Infinitv-Objektkomplement mit Subjektkontrolle (xcomp)

sent_nr = """
1 Haustiere 2 nsubj
2 hoffen 0 ROOT
3 wilde 4 amod
4 Tiere 6 obj
5 zu 6 mark
6 jagen 2 xcomp
"""
show_dep_trees(sent_nr)

### c) Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Haustiere denken wilde Tiere zu jagen ist falsch*


In [8]:
sent = 'Haustiere denken wilde Tiere zu jagen ist falsch'

#### Konstituentenanalyse:

In [9]:
grammar = nltk.CFG.fromstring(
    """
    S -> NP VP
    VP -> V SBAR
    SBAR -> COMP S
    COMP -> 
    S -> VP-INF VP
    VP-INF -> NP COMP V
    VP -> COP ADJ

    NP ->  N | ADJ N

    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'denken' | 'jagen'
    COMP -> 'zu' 
    COP -> 'ist'
    ADJ -> 'falsch'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

            S                                                   
    ┌───────┴─────┐                                              
    │             VP                                            
    │       ┌─────┴─────┐                                        
    │       │          SBAR                                     
    │       │     ┌─────┴───────────────┐                        
    │       │     │                     S                       
    │       │     │               ┌─────┴──────────────┐         
    │       │     │             VP-INF                 │        
    │       │     │          ┌────┴─────┬─────┐        │         
    NP      │     │          NP         │     │        VP       
    │       │     │     ┌────┴────┐     │     │    ┌───┴────┐    
    N       V    COMP  ADJ        N    COMP   V   COP      ADJ  
    │       │     │     │         │     │     │    │        │    
Haustiere denken ...  wilde     Tiere   zu  jagen ist     falsch



#### Dependenzanalyse:

In [10]:
#Objektsatz (ccomp) mit eingebettetem Infinitiv-Subjektsatz (csubj)

sent_nr = """
1 Haustiere 2 nsubj
2 denken 0 ROOT
3 wilde 4 amod
4 Tiere 6 obj
5 zu 6 mark
6 jagen 8 csubj
7 ist 8 cop
8 falsch 2 ccomp
"""
show_dep_trees(sent_nr)

## Aufgabe 2: Satzklassifikation und Syntaxregeln komplexer Sätze

#### Geben Sie (1.) zu der jeweils auftretenden komplexen Satzkonstruktion den Satz- bzw. Konstruktionstyp an. 

#### Geben Sie (2.) zu den CFG-Outputs des Stanford-Parsers (trainiert auf Penn-Treebank) anhand der Ableitungsbäume die Regeln der komplexen Satzkonstruktionen an.

#### Analysieren Sie (3.) anschließend die Dependenzstruktur der Sätze im UD-Schema.


- Penn-Treebank Tagset: http://surdeanu.cs.arizona.edu//mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html#Clause
- UD Dependency Tagset: https://universaldependencies.org/u/dep/all.html

---
### a)  *I believe that you win.*

In [12]:
sentence = "I believe that you win."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#Objektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#Komplementsatz

### 2. Konstituentenanalyse:

In [13]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP believe) (SBAR (IN that) (S (NP (PRP you)) (VP (VBP win))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                     
             │                        
             S                       
 ┌───────────┴────┬────────────────┐  
 │                VP               │ 
 │     ┌──────────┴───┐            │  
 │     │             SBAR          │ 
 │     │     ┌────────┴────┐       │  
 │     │     │             S       │ 
 │     │     │        ┌────┴───┐   │  
 NP    │     │        NP       VP  │ 
 │     │     │        │        │   │  
PRP   VBP    IN      PRP      VBP  . 
 │     │     │        │        │   │  
 I  believe that     you      win  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#VP -> VBP SBAR
#SBAR -> IN/COMP S

### 3. Dependenzanalyse:

In [14]:
sent_nr = """
1 I 2 nsubj  
2 believe 0 ROOT
3 that 5 mark
4 you 5 nsubj
5 win 2 ccomp
"""

show_dep_trees(sent_nr, style=2)

         believe(ROOT)                      
   ┌───────────┴───────────┐                 
   │                   win(ccomp)           
   │           ┌───────────┴──────────┐      
I(nsubj)   that(mark)             you(nsubj)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#believe -> win (ccomp)
#win -> that (mark)

### b) *I want to believe.*

In [15]:
sentence = "I want to believe."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#Infinitiv-Objektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#Komplementsatz

#### Zusatzfrage: Welche Form der Kontrolle liegt hier vor? (Subjekt- oder Objektkontrolle?)

In [ ]:
#Subjektkontrolle (Subjekt des Matrixsatzes ist semantisch Subjekt des Infinitivsatzes)

#### Welche Form der Kontrolle liegt in folgendem Satz vor: *Sie überreden ihn, nach München zu fahren?*


In [ ]:
#Objektkontrolle (Objekt des Matrixsatzes ist semantisch Subjekt des Infinitivsatzes)

#### Welche satzwertigen Funktionen können Infinitivkonstruktionen erfüllen? 

In [ ]:
# Objektsatz (xcomp): "sie plant (ein Buch zu lesen)"
# Subjektsatz (csubj): "(ein Buch zu lesen) ist gut"
# Adverbialsatz (advcl): "(um ein Buch zu lesen), macht sie früher Schluss"
# Attributsatz (acl): "sie hat den Plan, (ein Buch zu lesen)"

### 2. Konstituentenanalyse:

In [16]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP want) (S (VP (TO to) (VP (VB believe))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                
          │                   
          S                  
 ┌────────┼────────────────┐  
 │        VP               │ 
 │   ┌────┴────┐           │  
 │   │         S           │ 
 │   │         │           │  
 │   │         VP          │ 
 │   │    ┌────┴─────┐     │  
 NP  │    │          VP    │ 
 │   │    │          │     │  
PRP VBP   TO         VB    . 
 │   │    │          │     │  
 I  want  to      believe  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [61]:
#VP -> VBP S
#S -> VP (Satzregel für subjektlose Infinitivsätze)
#VP -> TO VP

### 3. Dependenzanalyse:

In [17]:
sent_nr = """
1 I 2 nsubj  
2 want 0 ROOT
3 to 4 mark
4 believe 2 xcomp
"""

show_dep_trees(sent_nr, style=2)

         want(ROOT)               
   ┌─────────┴────────────┐        
   │                believe(xcomp)
   │                      │        
I(nsubj)               to(mark)   



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [60]:
#want -> believe (xcomp)
#believe -> to (mark)

### c) *the man who knew too much*

In [18]:
sentence = "the man who knew too much"

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#Relativsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#Attributsatz

#### Zusatzfrage: Welche Attributsatztypen haben Sie in der Vorlesung kennengelernt? 

In [ ]:
# Relativsatz (`acl:relcl`)
# adnominaler Satz(`acl`): finite oder non-finite

### 2. Konstituentenanalyse:

In [19]:
tree_str = '(ROOT (FRAG (NP (NP (DT the) (NN man)) (SBAR (WHNP (WP who)) (S (VP (VBD knew) (ADJP (RB too) (JJ much))))))))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                        
             │                           
            FRAG                        
             │                           
             NP                         
     ┌───────┴─────────┐                 
     │                SBAR              
     │       ┌─────────┴────┐            
     │       │              S           
     │       │              │            
     │       │              VP          
     │       │    ┌─────────┴───┐        
     NP     WHNP  │            ADJP     
 ┌───┴───┐   │    │         ┌───┴────┐   
 DT      NN  WP  VBD        RB       JJ 
 │       │   │    │         │        │   
the     man who  knew      too      much



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#NP -> NP SBAR
#SBAR -> WHNP S

### 3. Dependenzanalyse:

In [20]:
sent_nr = """
1 the 2 det
2 man 0 ROOT
3 who 4 nsubj
4 knew 2 acl:relcl
5 too 6 advmod
6 much 4 advmod
"""

show_dep_trees(sent_nr, style=2)

         man(ROOT)                              
   ┌─────────┴─────────────┐                     
   │                knew(acl:relcl)             
   │         ┌─────────────┴─────────────┐       
   │         │                      much(advmod)
   │         │                           │       
the(det) who(nsubj)                 too(advmod) 



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#man -> knew (acl:relcl)
#knew -> who (nsubj)


### d) *They left, but they came back.* 

In [21]:
sentence = "They left, but they came back."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#koordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#keiner (koordinierte Sätze)

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#2 Hauptsätze

### 2. Konstituentenanalyse:

In [22]:
tree_str = '(ROOT (S (S (NP (PRP They)) (VP (VBD left))) (, ,) (CC but) (S (NP (PRP they)) (VP (VBD came) (ADVP (RB back)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                       
                   │                          
                   S                         
      ┌────────┬───┼─────────┬─────────────┐  
      │        │   │         S             │ 
      │        │   │    ┌────┴────┐        │  
      S        │   │    │         VP       │ 
 ┌────┴───┐    │   │    │    ┌────┴───┐    │  
 NP       VP   │   │    NP   │       ADVP  │ 
 │        │    │   │    │    │        │    │  
PRP      VBD   ,   CC  PRP  VBD       RB   . 
 │        │    │   │    │    │        │    │  
They     left  ,  but  they came     back  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#`S -> S , CC S .`

### 3. Dependenzanalyse:

In [23]:
sent_nr = """
1 they 2 nsubj
2 left 0 ROOT
3 but 5 cc
4 they 5 nsubj
5 came 2 conj
6 back 5 advmod
"""

show_dep_trees(sent_nr, style=2)

                    left(ROOT)                         
     ┌──────────────────┴───────────┐                   
     │                          came(conj)             
     │         ┌────────────────────┼───────────┐       
they(nsubj) but(cc)            they(nsubj) back(advmod)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#left -> came (conj)
#came -> but (cc)

### e) *This can't be right.*

In [24]:
sentence = "This can't be right."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#keines von beiden

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#keine Einbettung

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#(Satz mit komplexer VP)

### 2. Konstituentenanalyse:

In [25]:
tree_str = "(ROOT (S (NP (DT This)) (VP (MD ca) (RB n't) (VP (VB be) (ADJP (JJ right)))) (. .)))"
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                  
          │                     
          S                    
 ┌────────┼──────────────────┐  
 │        VP                 │ 
 │    ┌───┼────────┐         │  
 │    │   │        VP        │ 
 │    │   │    ┌───┴────┐    │  
 NP   │   │    │       ADJP  │ 
 │    │   │    │        │    │  
 DT   MD  RB   VB       JJ   . 
 │    │   │    │        │    │  
This  ca n't   be     right  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [59]:
#VP -> MD RB VP
#VP -> VB ADJP (Kopula-VP)

### 3. Dependenzanalyse:

HINWEIS Negation:
- `advmod`, vgl. https://universaldependencies.org/de/dep/advmod.html
- oder sprachspezifischer zusätzlicher `neg`-Marker: https://universaldependencies.org/docs/u/dep/neg.html

In [26]:
sent_nr = """
1 this 5 nsubj
2 ca 5 aux
3 n\'t 5 neg
4 be 5 cop
5 right 0 ROOT
"""

show_dep_trees(sent_nr, style=2)

            right(ROOT)                 
     ┌───────────┼─────────┬────────┐    
this(nsubj)   ca(aux)   n't(neg) be(cop)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#right -> ca (aux)
#right -> be (cop)


### f) *When the storm started, she was still outside.*

In [27]:
sentence = "When the storm started, she was still outside."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#Temporalsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#Adverbialsatz

### 2. Konstituentenanalyse:

In [28]:
tree_str = '(ROOT (S (SBAR (WHADVP (WRB When)) (S (NP (DT the) (NN storm)) (VP (VBD started)))) (, ,) (NP (PRP she)) (VP (VBD was) (ADVP (RB still)) (ADVP (RB outside))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                              ROOT                          
                               │                             
                               S                            
            ┌──────────────────┼────┬────────┬────────────┐  
           SBAR                │    │        │            │ 
  ┌─────────┴─────┐            │    │        │            │  
  │               S            │    │        VP           │ 
  │         ┌─────┴──────┐     │    │   ┌────┼──────┐     │  
WHADVP      NP           VP    │    NP  │   ADVP   ADVP   │ 
  │     ┌───┴─────┐      │     │    │   │    │      │     │  
 WRB    DT        NN    VBD    ,   PRP VBD   RB     RB    . 
  │     │         │      │     │    │   │    │      │     │  
 When  the      storm started  ,   she was still outside  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#S -> SBAR , NP VP .
#SBAR -> WHADVP S

### 3. Dependenzanalyse:

In [29]:
sent_nr = """
1 when 4 mark
2 the 3 det
3 storm 4 nsubj
4 started 6 advcl
5 she 6 nsubj
6 was 0 ROOT
7 still 6 advmod
8 outside 6 advmod 
"""

show_dep_trees(sent_nr, style=2)

                            was(ROOT)                                          
    ┌────────────┬──────────────┼─────────────────────────┐                     
    │            │              │                   started(advcl)             
    │            │              │            ┌────────────┴─────────────┐       
    │            │              │            │                     storm(nsubj)
    │            │              │            │                          │       
she(nsubj) still(advmod) outside(advmod) when(mark)                  the(det)  



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [58]:
#was -> started (advcl)
#started -> when (mark)


### g) *To go now would be a mistake.* 

In [30]:
sentence = "To go now would be a mistake."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#Infinitiv-Subjektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#Komplementsatz

### 2. Konstituentenanalyse:

In [31]:
tree_str = '(ROOT (S (S (VP (TO To) (VP (VB go) (ADVP (RB now))))) (VP (MD would) (VP (VB be) (NP (DT a) (NN mistake)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                            
                   │                               
                   S                              
     ┌─────────────┴────────┬───────────────────┐  
     S                      │                   │ 
     │                      │                   │  
     VP                     VP                  │ 
 ┌───┴───┐         ┌────────┴───┐               │  
 │       VP        │            VP              │ 
 │   ┌───┴───┐     │    ┌───────┴───┐           │  
 │   │      ADVP   │    │           NP          │ 
 │   │       │     │    │       ┌───┴─────┐     │  
 TO  VB      RB    MD   VB      DT        NN    . 
 │   │       │     │    │       │         │     │  
 To  go     now  would  be      a      mistake  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#S -> S VP .
#S -> VP
#VP -> TO VP

### 3. Dependenzanalyse:

In [32]:
sent_nr = """
1 to 2 mark
2 go 7 csubj
3 now 2 advmod
4 would 7 aux
5 be 7 cop
6 a 7 det
7 mistake 0 ROOT
"""

show_dep_trees(sent_nr, style=2)

                   mistake(ROOT)                               
    ┌─────────┬──────────┼────────────────────┐                 
    │         │          │                go(csubj)            
    │         │          │          ┌─────────┴──────────┐      
would(aux) be(cop)     a(det)    to(mark)           now(advmod)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [57]:
#mistake -> go (csubj)
#go -> to (mark)

## *Aufgabe 3 - Attributtypen und Attributfunktionen*

#### Analysieren Sie die folgende komplexe NP. Welche Attributtypen und -funktionen treten auf?
- *Die lang ersehnte Fahrt der Klasse 5b nach Paris, die sich aus irgendeinem Grunde immer wieder verzögert hatte, findet nun endlich statt.*

In [ ]:
# lang ersehnte: adnominaler Satz, acl
# nach Paris: Genitivattribut, nmod
# der Klasse 5b: Präpositionales Attribut, nmod
# die sich aus irgendeinem Grunde immer wieder verzögert hatte: attributiver Relativsatz, acl:relcl

In [34]:
sent_nr = """
1 Die 4 det
2 lang 3 advmod
3 ersehnte 4 acl
4 Fahrt 19 nsubj
5 der 6 det
6 Klasse 4 nmod 
7 5b 6 appos
8 nach 9 case
9 Paris 4 nmod 
10 die 17 nsbuj
11 sich 17 expl:pv
12 aus 14 case
13 irgendeinem 14 det
14 Grunde 17 obl
15 immer 16 advmod
16 wieder 17 advmod 
17 verzögert 4 acl:relcl
18 hatte 17 aux 
19 findet 0 ROOT
20 nun 19 advmod
21 endlich 19 advmod
22 statt 19 compound:prt
"""
show_dep_trees(sent_nr, style=0)

                                                                                         findet(ROOT)                                                                                                            
     ┌─────────────┬───────────────┬──────────────────────────────────────────────────────────┴────────────────────────┐                                                                                          
     │             │               │                                                                              Fahrt(nsubj)                                                                                   
     │             │               │           ┌───────────┬─────────────────────┬─────────────────────────┬───────────┴───────────────────────────┐                                                              
     │             │               │           │           │                     │                         │                                 verzögert(acl:   

## *Aufgabe 4 - Wortstellung im Nebensatz*

#### Betrachten Sie die folgenden beiden Sätze:
- *Peter sagt, er habe kein Geld.*
- *Peter sagt, dass er kein Geld habe.*

#### Der erste Nebensatz  ist durch eine Konjunktion eingeleitet, der zweite Nebensatz ist ohne Einleitewort. Leiten Sie daraus eine Regularität zur Stellung des finiten Verbs im Nebensatz ab.

### a) Wenn der Nebensatz durch eine Konjunktion eingeleitet wird, wo steht diese im Stellungsfeldermodell?

In [ ]:
# linke Satzklammer

### b) Von welchem Satzteil wird diese Position gefüllt, wenn die Konjunktion fehlt?

In [ ]:
# finites Verb

### c) Kann diese Position theoretisch auch leer bleiben?

In [44]:
# nein, der Satz wird dann ungrammatisch